<div style="text-align: right"> <b> Fachmodul | HS 2022 </b><br>
Windalarm <br>
<a href="mailto:pascal.schaback@ost.ch"> Pascal Schaback </a> </div>

# Meteomatics

In [31]:
import datetime as dt
import meteomatics.api as api

import json

import numpy as np

import math

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib

import folium
import ipyleaflet

## Lade Logindaten
Logindaten werden lokal gespeichert und werden nicht auf GIT gespeichert. (gitignore)

In [32]:
# Opening JSON file with my credentials

creds = open('mycredentials.json')
creds = json.load(creds)
USERNAME = creds["username"]
PASSWORD = creds["password"]

## Visualisiere Karte

In [33]:
def show_world_map(lon_lat):
    # Erstelle eine neue Folium-Karte mit dem Standort 0,0 als Zentrum
    m = folium.Map(location=lon_lat, tiles="Stamen Terrain", zoom_start=13)

    # Füge einen Marker an den angegebenen Längen- und Breitengrad-Koordinaten zur Karte hinzu
    #folium.Marker(lon_lat).add_to(m)

    # Zeige die Karte an
    return m

In [34]:
parameter = ["Wind", "Temperatur", "Druck"]
position = ([47.1287, 9.21605])

map = show_world_map(position)
map

## Lade Winddaten
API von Meteomatics


In [35]:
def get_grid_data(date_time, parameter, resolution, model="Mix"):

    lat_N, lon_W = [47.157400722577655, 9.074223132773003]
    lat_S, lon_E = [47.088922131607625, 9.35639395703287]

    result = dict()
    for param in parameter:
        try:
            df_grid, url = api.query_grid(date_time, param, lat_N=lat_N, lon_W=lon_W, lat_S=lat_S, lon_E=lon_E, res_lat=resolution, res_lon=resolution,
                                    username=USERNAME, password=PASSWORD, model=model)
            result[param] = df_grid
        except Exception as e:
            print("Failed, the exception is {}".format(e))
            result[param] = None
    return result

In [36]:
date_time = dt.datetime(day=20, month=10, year=2022, hour=11, minute=0, second=0, microsecond=0)
resolution = 0.005
model="Mix"
parameter =("wind_dir_10m:d", "wind_speed_10m:kmh")

result = get_grid_data(date_time, parameter, resolution)
result.keys()

url https://api.meteomatics.com/2022-10-20T11:00:00+00:00/wind_dir_10m:d/47.157400722577655,9.074223132773003_47.088922131607625,9.35639395703287:0.005,0.005/bin?connector=python_v2.8.1&model=Mix
url https://api.meteomatics.com/2022-10-20T11:00:00+00:00/wind_speed_10m:kmh/47.157400722577655,9.074223132773003_47.088922131607625,9.35639395703287:0.005,0.005/bin?connector=python_v2.8.1&model=Mix


dict_keys(['wind_dir_10m:d', 'wind_speed_10m:kmh'])

## Zeichne Pfeil in Folium
Es gibt zur Zeit leider kein Plugin für Folium, die dies macht.

In [37]:
def rotate(origin, point, angle):
    angle = math.radians(angle)
    
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return np.array((qx, qy))

In [38]:
def add_arrow_to_featuregroup(feature_group, pos, dir, length, color):
    
    # 0° Pfeil zeigt gegen Süden
    # 270° Pfeil zeigt gegen Osten
    
    dir = dir+180

    x0 = np.array([0, 0])
    x1 = np.array([length, 0])

    x2_1 = x1-[0.3*length, -0.2*length]
    x2_2 = x1-[0.3*length, 0.2*length]

    x1_rot = rotate(x0, x1, dir)

    feature_group.add_child(folium.PolyLine(locations=[pos, pos+x1_rot], weight=2, color=color))
    feature_group.add_child(folium.PolyLine(locations=[pos+x1_rot, pos+rotate(x0, x2_1, dir)], weight=2, color=color))
    feature_group.add_child(folium.PolyLine(locations=[pos+x1_rot, pos+rotate(x0, x2_2, dir)], weight=2, color=color))
    return feature_group

In [39]:
def add_wind_markers(map, result):
    wind_speed_map = result['wind_speed_10m:kmh']
    wind_dir_map = result['wind_dir_10m:d']

    size=0.01
    
    #normalize arrow length
    wind_speed_norm = wind_speed_map / wind_speed_map.max().max() * size

    #normalize item number values to colormap
    norm = matplotlib.colors.Normalize(vmin=0, vmax=80)

    #colormap possible values = viridis, jet, spectral
    rgba_color = cm.nipy_spectral(norm(400),bytes=True)

    feature_group_wind = folium.FeatureGroup("Wind")
    
    for idx in wind_dir_map.index:
        for col in wind_dir_map.columns:
            #print("wind_heading", wind_dir_map.loc[idx,col], "\t wind speed:", wind_speed_map.loc[idx,col], "\t @", idx, col)
            
            color = cm.nipy_spectral(norm(wind_speed_map.loc[idx,col]))
            try:
                feature_group_wind = add_arrow_to_featuregroup(feature_group_wind, ([idx,col]), wind_dir_map.loc[idx,col], wind_speed_norm.loc[idx,col], matplotlib.colors.rgb2hex(color,keep_alpha=True))
            except ValueError:
                print("NAN")
    feature_group_wind.add_to(map)
    folium.LayerControl().add_to(map)
    return map

In [40]:
map = show_world_map(position)
new_map = add_wind_markers(map, result)
new_map

# Interaktion
Zur Evaluierung der Daten sollen folgende Parameter interaktiv geändert werden können:
- Datum und Zeit
- dargestellter Parameter
- Kartenausschnitt und Auflösung

In [41]:
import ipywidgets as widgets

In [42]:
date = widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)

In [43]:
hour = widgets.IntSlider(
    value=12,
    min=8,
    max=16,
    step=1,
    description='Stunde:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [44]:
minute = widgets.IntSlider(
    value=0,
    min=0,
    max=50,
    step=5,
    description='Minute:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [45]:
def show_wind_map(model="Mix"):
    try:
        date_time = dt.datetime(day=date.value.day, month=date.value.month, year=date.value.year, hour=hour.value, minute=minute.value)
        resolution = 0.005
            
        parameter =("wind_dir_10m:d", "wind_speed_10m:kmh")
        
        try:
            result = get_grid_data(date_time, parameter, resolution, model)
            map = add_wind_markers(show_world_map(position), result)
            return map
        except AttributeError:
            print()
    except AttributeError:
        print("please choose Date")

In [46]:
datetime_picker = widgets.HBox([date, hour, minute])
datetime_picker

Leider wird die Karte nicht interaktiv angezeigt. Kann nicht in dem Output dargestellt werden. Alternativ jedes mal nach dem Ändern der Zeit folgenden Code kurz ausführen:

In [51]:
date.value

datetime.date(2022, 12, 11)

In [52]:
show_wind_map(model="mix-obs")

url https://api.meteomatics.com/2022-12-11T12:00:00+00:00/wind_dir_10m:d/47.157400722577655,9.074223132773003_47.088922131607625,9.35639395703287:0.005,0.005/bin?connector=python_v2.8.1&model=mix-obs
url https://api.meteomatics.com/2022-12-11T12:00:00+00:00/wind_speed_10m:kmh/47.157400722577655,9.074223132773003_47.088922131607625,9.35639395703287:0.005,0.005/bin?connector=python_v2.8.1&model=mix-obs
